# Let's Train ChefGPT

Let's fine tune our own ChefGPT model. 

## Let's Train

In [1]:
from datasets import load_from_disk

train_tokenized = load_from_disk('./tokenized_train_dataset_10k_v1')
val_tokenized = load_from_disk('./tokenized_test_dataset_10k_v1')

In [2]:
train_tokenized.shape

(10000, 3)

In [3]:
val_tokenized.shape

(2000, 3)

## Let's Train

In [4]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, TrainingArguments, Trainer

In [5]:
# Configure the model
config = T5Config.from_pretrained('t5-base')

# Initialize the model
model = T5ForConditionalGeneration.from_pretrained('t5-base', config=config)

# Disable caching for the model, prevents cache error message during the training!
model.config.use_cache = False

In [6]:
# Configure training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    auto_find_batch_size=True,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch',
    logging_dir='./logs',
    gradient_accumulation_steps=2,
#     gradient_checkpointing=True,
    learning_rate=1e-4,
    warmup_steps=100,
    weight_decay=0.01,
    fp16=True,
    load_best_model_at_end=True,
    optim="adamw_torch",
    adam_beta1=0.85    
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized
)

In [7]:
# import torch
# torch.cuda.empty_cache()

# Fine-tune the model
trainer.train()

/home/datascience/conda/pytorch110_p38_gpu_v1/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,3.078900,2.008936
2,2.151400,1.919985
2,2.142400,1.917045


/home/datascience/conda/pytorch110_p38_gpu_v1/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/datascience/conda/pytorch110_p38_gpu_v1/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=936, training_loss=2.457566872621194, metrics={'train_runtime': 1257.7518, 'train_samples_per_second': 23.852, 'train_steps_per_second': 0.744, 'total_flos': 1.823950673805312e+16, 'train_loss': 2.457566872621194, 'epoch': 3.0})

In [8]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_t5_recipes_base_10k_v1")